# 새싹 해커톤
## 나란히 - 장애인 인식 개선을 위한 생성형AI

### 1. PDF를 읽어보기

#### 1-1) PyPDF2, PyMuPDF

In [17]:
import PyPDF2

def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
    return text

import fitz  # PyMuPDF


def read_pdf_with_pymupdf(file_path):
    doc = fitz.open(file_path)
    text = ''
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)  # 페이지 불러오기
        text += page.get_text()  # 페이지에서 텍스트 추출
    return text


pdf_text = read_pdf_with_pymupdf('./dataset/[모듈형 교육PPT] 시각장애.pdf')
print(pdf_text)

In [18]:
if __name__ == "__main__":
    file_path = './dataset/[모듈형 교육PPT] 시각장애.pdf'

    pdf_text = read_pdf(file_path)
    print(pdf_text)

#### OCR인식 없이는 잘 읽히지 않는 것 같다.

python에서 지원하는 PDF reader를 사용해보았지만...  
드래그로 긁을 수 있는 형식이 아니라 그런지 잘 읽히진 않는다.

### 1-2) OCR reader (pytesseract)

In [21]:
"""
Requirements: `pip install pytesseract pdf2image pillow`
"""

# OCR을 사용

import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os

# Tesseract 경로 설정 (윈도우의 경우)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# PDF 파일 경로 설정
pdf_path = './dataset/[모듈형 교육PPT] 시각장애.pdf'

# PDF를 이미지로 변환, 해상도 300dpi(고화질)로 조정
pages = convert_from_path(pdf_path, 300) 

# 각 페이지에서 텍스트 추출
def ocr_from_pdf(pages):
    text = ''
    for num, page in enumerate(pages):
        # 이미지를 일시적으로 저장
        temp_image_path = f"temp_page_{num+1}.png"
        page.save(temp_image_path, 'PNG')
        
        # 이미지에서 텍스트 추출
        page_text = pytesseract.image_to_string(Image.open(temp_image_path), lang='kor+eng')
        text += page_text
        
        # 일시적 이미지 파일 삭제
        os.remove(temp_image_path)
        
    return text

In [ ]:
# 텍스트 추출
pdf_text = ocr_from_pdf(pages)
print(pdf_text)

AttributeError: module 'fitz' has no attribute '__version__'

In [14]:
pdf_text

'장매유형별 모듈\n\nContents\n\n0. 장애이해하7|(공통)\n1. 일터 엿보기\n\n2. 미해하기\n\n4. 7|1억해주세요/\n\n5. 마무리\n\n0. 장애 이해하7|                                                                          한국장애인고용공단 "에\n\n오랫동안 일상 및 사희생활에\n상당한 제약을 받는 자\n\n장애인이란 신체\n장기간에 걸쳐 직\n상당한 제약을 받는 자\n\nor7|\n\n애\n\n장애 이\n\n0.\n\n분류됩니다.\n\n은 15개로\n\nrt |\nSo\neS\n\nOo\nTT\n\nOH\n\n| ZI\noO\n\nM 우리나라으\n\n=\nfo}\nLo\n\n~\n\n>지체장애\n\n{=|\n|\n\n외\n\nㆍ시각장애\n\n장애\n\n인\n이라고 해도 개별사례별로 다를 수 있습니다.\n\n고\n\na\n쁘\nal\n=\n10\nKid\n제\nult\nOl\noe\n아\n!!\n\n0. 장애 이해하7|                                                                          한국장애인고용공단 "에\n\n장애등급제 HN\n\n복/|/ 비스 NAS 위한 장애인\n장매등급은 장애정도로 변경\n\nXt\n=\n\nㆍ의학적 기준(1~6급)                   ㆍ심한 장애인(1~5급)\nㆍ심하지 않은 장애인(4~<급)\n\n0. 장애 이해하7|                                                                          한국장애인고용공단 "에\n\nADA ©\n\n인등록현황 / 2019 122! 기준)\n\n0. 장애 이해하기                                                                            한국장아인고8공단 SP\n\n출생시원인\n14%\n선천적원인\n51%\n\n후천적 

In [16]:
# 텍스트를 txt 파일로 저장
def save_text_to_file(text, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

# 저장할 파일 경로 설정
txt_file_path = 'extracted_text.txt'
save_text_to_file(pdf_text, txt_file_path)
print(f"Text saved to {txt_file_path}")

Text saved to extracted_text.txt


읽히긴 하지만 오타도 많고 의미를 알아보기가 쉽지 않았다.  
이미지 전처리를 통해 

In [23]:
import cv2
import numpy as np
from PIL import Image

# 이미지 전처리
def preprocess_image(image):
    # 이미지를 그레이스케일로 변환
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    # 이진화
    _, binary_image = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # 노이즈 제거
    processed_image = cv2.medianBlur(binary_image, 3)
    return Image.fromarray(processed_image)


# 각 페이지에서 텍스트 추출
def ocr_from_pdf(pages):
    text = ''
    for num, page in enumerate(pages):
        # 이미지를 일시적으로 저장
        temp_image_path = f"temp_page_{num+1}.png"
        processed_page = preprocess_image(page)
        processed_page.save(temp_image_path, 'PNG')
        
        # 이미지에서 텍스트 추출
        page_text = pytesseract.image_to_string(Image.open(temp_image_path), lang='kor+eng')
        text += page_text
        
        # 일시적 이미지 파일 삭제
        os.remove(temp_image_path)
        
    return text


In [24]:
pdf_text_preprocessed = ocr_from_pdf(pages)
print(pdf_text_preprocessed)

txt_file_path = 'preprocessed_OCR.txt'
save_text_to_file(pdf_text_preprocessed, txt_file_path)
print(f"Text saved to {txt_file_path}")

장매유형별 모듈

한국장애인고용공단 FP

Contents

0. 장애이해하7|(공통)
1. 일터 엿보기

2. 미해하기

3. 함께 일하7|

4. 71억해주세요/

5. 마무리

0. St0H 이해하기                                                                          한국장야인고8공단 FL

장매인복지법

장애인은 신체적ㆍ정신적 장애로 인하여

(0
오랫동안 일상 및 사회생활에
상당한 제약을 받는 자

장애인이란 신체 또는 정
장기간에 걸쳐 직업생활에
상당한 제약을 받는 자

0. 장애 이해하기
한국장애인고용공단

ㆍ지체장애
>뇌병변장애                                             끼적

» 시각장애                                                강매

0. Zt0H 이해하기                                                                          eagqensan AP

장애등급제 폐지

복/끼서비스 NAS 위한 장애인

MOSES 장매정도로 변경

ㆍ의학적 기준(1~6급)                       ㆍ심한 장애인(1~3급)
ㆍ심하지 않은 장애인(4~6급)

0. St0H 이해하기                                                                         한국장애인고8공단 FP

AAA @

록현황 / 20194 128 기준)

0. St0H 이해하기                                                                          한국장야인고8공단 FL

출생시원인
14%

원인불명 선전적원인 |
54% —N%

후천적 질환

56.0%

출처 : 2017년 장애인 실태조사

누구도 장애인이 될 가능성에서 ASSN 않습니다

0. 장애 이해하기    

In [29]:
"""
Requirements: `pip install pyspellchecker`
"""

from spellchecker import SpellChecker

spell = SpellChecker(language='ko')

def correct_spelling(text):
    corrected_text = []
    words = text.split()
    for word in words:
        corrected_word = spell.correction(word)
        corrected_text.append(corrected_word)
    return ' '.join(corrected_text)

# 텍스트 추출 후 철자 교정
# pdf_text = ocr_from_pdf(pages)
corrected_text = correct_spelling(pdf_text_preprocessed)
print(corrected_text)

ValueError: The provided dictionary language (ko) does not exist!

In [37]:
"""
Requirements: `pip install korpora`
"""

from Korpora import Korpora

# 사용할 데이터셋 다운로드
Korpora.fetch('kowikitext')

# 데이터 로드
kowikitext = Korpora.load('kowikitext')

# 단어 목록 추출
words = set()

# KoWikiText 데이터에서 단어 추출
for text in kowikitext.train:
    words.update(text.split())

# 단어 목록을 파일로 저장
with open('korean_words.txt', 'w', encoding='utf-8') as f:
    for word in words:
        f.write(f"{word}\n")


[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.train.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.train
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.test.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.test
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.dev.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.dev

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repos

AttributeError: 'SentencePair' object has no attribute 'split'

In [39]:
from Korpora import Korpora

# 사용할 데이터셋 다운로드
Korpora.fetch('kowikitext')

# 데이터 로드
kowikitext = Korpora.load('kowikitext')

[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.train.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.train
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.test.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.test
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.dev.zip
[Korpora] Corpus `kowikitext` is already installed at C:\Users\dm705\Korpora\kowikitext\kowikitext_20200920.dev

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repos

In [40]:
import re

# 한글 단어만 추출하기 위한 정규식 패턴
hangul_pattern = re.compile(r'[가-힣]+')

# 단어 목록 추출
words = set()

# KoWikiText 데이터에서 단어 추출
for sentence in kowikitext.train:
    for word in sentence.text.split():
        if hangul_pattern.fullmatch(word):
            words.add(word)

In [41]:
# 단어 목록을 파일로 저장
with open('korean_words.txt', 'w', encoding='utf-8') as f:
    for word in words:
        f.write(f"{word}\n")

# 결과 확인
print(f"총 {len(words)}개의 한국어 단어가 추출되었습니다.")

총 4426308개의 한국어 단어가 추출되었습니다.


In [43]:
from spellchecker import SpellChecker

# 한국어 단어 목록 파일 경로
dictionary_path = 'korean_words.txt'
index_path = 'korean_dictionary.gz'

# 한국어 단어 목록을 불러와서 인덱스 파일 생성
spell = SpellChecker(language=None)
with open(dictionary_path, 'r', encoding='utf-8') as file:
    words = file.read().splitlines()
    spell.word_frequency.load_words(words)

# 인덱스 파일로 저장
spell.export(index_path, gzipped=True)


In [45]:
from spellchecker import SpellChecker

# 생성된 인덱스 파일 설정
spell = SpellChecker(language=None, local_dictionary='korean_dictionary.gz')

# 철자 교정 예제
text = "장애인 인식 개선을 위한 방법은 무엇인가요?"
words = text.split()
corrected_words = [spell.correction(word) for word in words]
print(corrected_words)
corrected_text = ' '.join(corrected_words)
print(corrected_text)


['장애인', '인식', '개선을', '위한', '방법은', '무엇인가요']
장애인 인식 개선을 위한 방법은 무엇인가요
